In [58]:
import os
import glob
import pandas as pd

In [59]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20171016_gRNA1_Pax7_Sox9/sec/pax7counts'

In [60]:
# Add experiment date here to apply to dataframe
expt_date = '20171016'
treatment = 'SMPD3gRNA1'
result_type = 'Pax7Counts'

path = os.path.abspath('')+'/pax7csv/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Parse Image name
    df['Embryo'], df['Somites'], df['Section'] = zip(*df['Image'].map(lambda x: x.split('_')))

    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Date,Image,Embryo,Somites,Section
0,CntlSide,15,1549.477,103.298,23.450,255,26341.109,20171016,Emb9_9ss_sec3,Emb9,9ss,sec3
1,ExptSide,32,2282.116,71.316,20.986,255,18185.615,20171016,Emb9_9ss_sec3,Emb9,9ss,sec3
0,CntlSide,22,1805.576,82.072,19.252,255,20928.269,20171016,Emb9_9ss_sec2,Emb9,9ss,sec2
1,ExptSide,38,2575.007,67.763,22.324,255,17279.654,20171016,Emb9_9ss_sec2,Emb9,9ss,sec2
0,CntlSide,43,2829.097,65.793,19.609,255,16777.201,20171016,Emb9_9ss_sec1,Emb9,9ss,sec1


In [61]:
# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Embryo', 'Slice'])['Count']).mean())
mean_sections
# Extract means for Cntl and Expt counts
pax7_cntl = mean_sections.xs('CntlSide', level='Slice')['Count'] 
pax7_expt = mean_sections.xs('ExptSide', level='Slice')['Count']

# Generate ratios as Expt/Cntl
pax7_ratios = pd.DataFrame(pax7_expt / pax7_cntl)
pax7_ratios.columns = ['Expt/Cntl Pax7']

In [62]:
# Combine processed values into single dataframe and output as csv file
pax7_cntl = pd.DataFrame(pax7_cntl)
pax7_cntl.columns = ['Cntl Pax7']
pax7_expt = pd.DataFrame(pax7_expt)
pax7_expt.columns = ['Expt Pax7']

results = pd.concat([pax7_cntl, pax7_expt, pax7_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')